In [ ]:
!pip install tfx
!pip install tensorflow_model_analysis
!pip install tensorflow
!pip install google-cloud-bigquery
!pip install packaging==21.3
!pip install fastapi
!pip install kaleido
!pip install python-multipart
!pip install uvicorn
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
ml-pipelines-sdk 1.14.0 requires packaging<21,>=20, but you have packaging 21.3 which is incompatible.
pandas-gbq 0.19.2 requires google-cloud-bigquery!=2.4.*,<4.0.0dev,>=3.3.5, but you have google-cloud-bigquery 2.34.4 which is incompatible.
tfx 1.14.0 requires packaging<21,>=20, but you have packaging 21.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.20.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
ml-pipelines-sdk 1.14.0 requires packaging<21,>=20, but you have packaging 21.3 which is incompatible.
tensorflow 2.13.1 requires typing-extensions<4.6.0,>=3.

In [ ]:
import os
import pandas as pd
import tensorflow_model_analysis as tfma

In [ ]:
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

In [ ]:
import os
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from keras.utils import plot_model
from components import initialize_components
from pipeline import initialize_local_pipeline
import time

start = time.time()
PIPELANE_NAME = "pipeline-diabetes"

# Pipeline inputs
DATA_ROOT = "dataset"
TRANSFORM_MODULE_FILE = "transform.py"
TRAINER_MODULE_FILE = "trainer.py"

# Pipeline outputs
OUTPUT_BASE = "luaran"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELANE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

components = initialize_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    training_module=TRAINER_MODULE_FILE,
    training_steps=5000,
    eval_steps=1000,
    serving_model_dir=serving_model_dir
)

section = initialize_local_pipeline(components, pipeline_root)
BeamDagRunner().run(section)
end = time.time()

print('The whole process takes up to: ', (end-start)/60, 'minutes')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 smoking_history_xf (InputL  [(None, 4)]                  0         []                            
 ayer)                                                                                            
                                                                                                  
 gender_xf (InputLayer)      [(None, 3)]                  0         []                            
                                                                                                  
 age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 hypertension_xf (InputLaye  [(None, 1)]                  0         []                      

/usr/local/lib/python3.10/dist-packages/tensorflow_model_analysis/metrics/confusion_matrix_metrics.py:539: RuntimeWarning: invalid value encountered in divide
  recall = tp / (tp + fn)
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


The whole process takes up to:  7.830762275060018 minutes


In [ ]:
!zip -r /content/output.zip /content/luaran

  adding: content/luaran/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/statistics/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/statistics/3/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/statistics/3/Split-eval/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/statistics/3/Split-eval/FeatureStats.pb (deflated 72%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/statistics/3/Split-train/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/statistics/3/Split-train/FeatureStats.pb (deflated 72%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/.system/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/.system/executor_execution/ (stored 0%)
  adding: content/luaran/pipeline-diabetes/StatisticsGen/.system/executor_execution

In [ ]:
!zip -r /content/outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/pipeline-diabetes/ (stored 0%)
  adding: content/outputs/pipeline-diabetes/metadata.sqlite (deflated 94%)
